In [1]:
%load_ext tensorboard.notebook

In [2]:
!rm -rf ./logs/ 

In [3]:
import tensorflow as tf
import datetime

In [4]:
from tensorflow.keras import datasets, layers, models


In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [6]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [7]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0


In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.fit(train_images, train_labels, epochs=15, callbacks=[tensorboard_callback])

Epoch 1/15
60000/60000 [==============================] - 13s 212us/sample - loss: 0.1539 - accuracy: 0.9523
Epoch 2/15
60000/60000 [==============================] - 12s 204us/sample - loss: 0.0478 - accuracy: 0.9852
Epoch 3/15
60000/60000 [==============================] - 13s 220us/sample - loss: 0.0343 - accuracy: 0.9890
Epoch 4/15
60000/60000 [==============================] - 13s 215us/sample - loss: 0.0263 - accuracy: 0.9918
Epoch 5/15
60000/60000 [==============================] - 13s 212us/sample - loss: 0.0220 - accuracy: 0.9931
Epoch 6/15
60000/60000 [==============================] - 13s 221us/sample - loss: 0.0165 - accuracy: 0.9945
Epoch 7/15
60000/60000 [==============================] - 13s 221us/sample - loss: 0.0147 - accuracy: 0.9954
Epoch 8/15
60000/60000 [==============================] - 15s 252us/sample - loss: 0.0132 - accuracy: 0.9960
Epoch 9/15
60000/60000 [==============================] - 13s 209us/sample - loss: 0.0104 - accuracy: 0.9964
Epoch 10/15
60000/6

In [11]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 31197), started -1 day, 23:15:58 ago. (Use '!kill 31197' to kill it.)